# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [5]:
import pandas as pd, re

def q1_read_table(html: str) -> pd.DataFrame:
    tables = pd.read_html(html)
    for t in tables:
        if t.shape[1] >= 3:
            df = t.copy()
            break
    else:
        raise ValueError("No table with >=3 columns found.")

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x) != "nan"]).strip() for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]

    rename_map = {}
    for c in df.columns:
        if re.search(r'\bcountry\b', c, re.I): rename_map[c] = "Country"
        elif re.search(r'alpha[-\s_]*2', c, re.I): rename_map[c] = "Alpha-2"
        elif re.search(r'alpha[-\s_]*3', c, re.I): rename_map[c] = "Alpha-3"
        elif re.search(r'\bnumeric\b|\biso\s*3166-1\s*numeric\b', c, re.I): rename_map[c] = "Numeric"
    return df.rename(columns=rename_map)

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = out[c].astype(str).str.strip()
    out["Alpha-2"] = out["Alpha-2"].astype(str).str.upper()
    out["Alpha-3"] = out["Alpha-3"].astype(str).str.upper()
    out["Numeric"] = pd.to_numeric(out["Numeric"], errors="coerce").astype("Int64")
    out = out[out["Country"].astype(str).str.strip().ne("")].reset_index(drop=True)
    return out

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values("Numeric", ascending=False, na_position="last").head(top).reset_index(drop=True)


In [6]:
Q1_URL = "https://www.iban.com/country-codes"
html = fetch_html(Q1_URL)
raw = q1_read_table(html)
clean = q1_clean(raw)
clean.to_csv("data_q1.csv", index=False)
display(q1_sort_top(clean))


/tmp/ipython-input-853688934.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


,Country,Alpha-2,Alpha-3,Numeric
0,Zambia,ZM,ZMB,894
1,Yemen,YE,YEM,887
2,Samoa,WS,WSM,882
3,Wallis and Futuna,WF,WLF,876
4,Venezuela (Bolivarian Republic of),VE,VEN,862
5,Uzbekistan,UZ,UZB,860
6,Uruguay,UY,URY,858
7,Burkina Faso,BF,BFA,854
8,Virgin Islands (U.S.),VI,VIR,850
9,United States of America (the),US,USA,840


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [7]:
from bs4 import BeautifulSoup

def _extract_digits(text: str) -> int:
    if not isinstance(text, str): return 0
    m = re.search(r'\d+', text.replace(',', ''))
    return int(m.group()) if m else 0

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    items = []
    for row in soup.select("tr.athing"):
        rank = _extract_digits((row.select_one("span.rank") or {}).get("text", "") or (row.select_one("span.rank") or "").get_text(strip=True))
        a = row.select_one(".titleline a")
        title, link = (a.get_text(strip=True), a.get("href", "")) if a else ("", "")
        subtext = row.find_next_sibling("tr").select_one(".subtext") if row.find_next_sibling("tr") else None
        score = subtext.select_one(".score").get_text(strip=True) if subtext and subtext.select_one(".score") else ""
        user = subtext.select_one("a.hnuser").get_text(strip=True) if subtext and subtext.select_one("a.hnuser") else ""
        comments = 0
        if subtext and subtext.find_all("a"):
            comments = _extract_digits(subtext.find_all("a")[-1].get_text(strip=True).lower())
        items.append({"rank": rank, "title": title, "link": link, "points": _extract_digits(score), "comments": comments, "user": user})
    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    out = df.fillna("")
    for col in ["rank", "points", "comments"]:
        out[col] = pd.to_numeric(out[col], errors="coerce").fillna(0).astype(int)
    for col in ["title", "link", "user"]:
        out[col] = out[col].astype(str).str.strip()
    return out

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values("points", ascending=False, na_position="last").head(top).reset_index(drop=True)


In [8]:
Q2_URL = "https://news.ycombinator.com/"
html = fetch_html(Q2_URL)
raw = q2_parse_items(html)
clean = q2_clean(raw)
clean.to_csv("data_q2.csv", index=False)
display(q2_sort_top(clean))

,rank,title,link,points,comments,user
0,6,Mr TIFF,https://inventingthefuture.ghost.io/mr-tiff/,840,113,speckx
1,22,I’m worried that they put co-pilot in Excel,https://simonwillison.net/2025/Nov/5/brenda/,252,174,isaacfrond
2,15,UPS plane crashes near Louisville airport,https://avherald.com/h?article=52f5748f&opt=0,244,222,jnsaff2
3,17,Bluetui – A TUI for managing Bluetooth on Linux,https://github.com/pythops/bluetui,217,78,birdculture
4,14,RISC-V takes first step toward international I...,https://riscv.org/blog/risc-v-jtc1-pas-submitter/,205,76,jrepinc
5,21,Apple’s Persona technology uses Gaussian splat...,https://www.cnet.com/tech/computing/apple-talk...,182,83,dmarcos
6,16,Hypothesis: Property-Based Testing for Python,https://hypothesis.readthedocs.io/en/latest/,173,100,lwhsiao
7,8,SPy: An interpreter and compiler for a fast st...,https://antocuni.eu/2025/10/29/inside-spy-part...,154,61,og_kalu
8,7,iOS 26.2 to allow third-party app stores in Ja...,https://www.macrumors.com/2025/11/05/ios-26-2-...,151,90,tosh
9,29,Grayskull: A tiny computer vision library in C...,https://github.com/zserge/grayskull,147,13,gurjeet
